In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
sales = pd.read_csv("sales_weekly.csv")          # columns: id, item_id, dept_id, cat_id, store_id, state_id, date, units_sold
calendar = pd.read_csv("calendar_weekly.csv")    # columns: date, wm_yr_wk, event_name_1, event_type_1, event_name_2, event_type_2
prices = pd.read_csv("prices_weekly.csv")   # columns: store_id, item_id, wm_yr_wk, sell_price

In [ ]:
import pandas as pd
from pathlib import Path

output_dir = Path("Data/star_schema/")
output_dir.mkdir(parents=True, exist_ok=True)

# --- DIM_CALENDAR ---
# Giả định calendar có cột: wm_yr_wk, main_event_name, main_event_type
dim_calendar = calendar.rename(columns={
    "wm_yr_wk": "week_id",
    "main_event_name": "event_name",
    "main_event_type": "event_type"
})

# Tách năm và số tuần từ mã tuần (vd: 2011-15 → year=2011, week=15)
dim_calendar["year"] = dim_calendar["week_id"].astype(str).str[:4].astype(int)
dim_calendar["week_of_year"] = dim_calendar["week_id"].astype(str).str[-2:].astype(int)

# Thêm các cột placeholder (vì không có trong dữ liệu gốc)
dim_calendar["weekday"] = None
dim_calendar["month"] = None
dim_calendar["snap_CA"] = False
dim_calendar["snap_TX"] = False
dim_calendar["snap_WI"] = False

# --- DIM_ITEM ---
# Lấy giá trung bình theo item_id từ bảng giá
dim_item = (
    prices.groupby("item_id", as_index=False)
    .agg({"avg_sell_price": "mean"})
    .rename(columns={"avg_sell_price": "price"})
)

# Tạo thêm thông tin mô tả
dim_item["currency"] = "USD"
dim_item["dept_id"] = sales.groupby("item_id")["dept_id"].first().values
dim_item["cat_id"] = sales.groupby("item_id")["cat_id"].first().values
dim_item["item_name"] = dim_item["item_id"]  # giả định tạm item_name = item_id

# --- DIM_STORE ---
dim_store = sales[["store_id", "state_id"]].drop_duplicates().reset_index(drop=True)
dim_store["store_name"] = dim_store["store_id"]
dim_store["type"] = None
dim_store["size"] = None

# --- DIM_STATE ---
dim_state = sales[["state_id"]].drop_duplicates().reset_index(drop=True)
dim_state["state_name"] = dim_state["state_id"]
dim_state["region"] = None

# --- FACT_SALES ---
fact_sales = sales.merge(
    prices,
    on=["store_id", "item_id", "wm_yr_wk"],
    how="left"
)

fact_sales["revenue"] = fact_sales["weekly_units_sold"] * fact_sales["avg_sell_price"]
fact_sales = fact_sales.rename(columns={"wm_yr_wk": "week_id"})
fact_sales = fact_sales[[
    "week_id", "item_id", "store_id", "weekly_units_sold", "revenue"
]]

# --- SAVE ALL TO FILES ---
dim_calendar.to_csv(output_dir / "dim_calendar.csv", index=False)
dim_item.to_csv(output_dir / "dim_item.csv", index=False)
dim_store.to_csv(output_dir / "dim_store.csv", index=False)
dim_state.to_csv(output_dir / "dim_state.csv", index=False)
fact_sales.to_csv(output_dir / "fact_sales.csv", index=False)

print(f"✅ All star schema tables generated and saved to '{output_dir}' directory.")
 pandas as pd
from pathlib import Path

output_dir = Path("Data/star_schema/")
output_dir.mkdir(parents=True, exist_ok=True)

# --- DIM_CALENDAR ---
# Giả định calendar có cột: wm_yr_wk, main_event_name, main_event_type
dim_calendar = calendar.rename(columns={
    "wm_yr_wk": "week_id",
    "main_event_name": "event_name",
    "main_event_type": "event_type"
})

# Tách năm và số tuần từ mã tuần (vd: 2011-15 → year=2011, week=15)
dim_calendar["year"] = dim_calendar["week_id"].astype(str).str[:4].astype(int)
dim_calendar["week_of_year"] = dim_calendar["week_id"].astype(str).str[-2:].astype(int)

# Thêm các cột placeholder (vì không có trong dữ liệu gốc)
dim_calendar["weekday"] = None
dim_calendar["month"] = None
dim_calendar["snap_CA"] = False
dim_calendar["snap_TX"] = False
dim_calendar["snap_WI"] = False

# --- DIM_ITEM ---
# Lấy giá trung bình theo item_id từ bảng giá
dim_item = (
    prices.groupby("item_id", as_index=False)
    .agg({"avg_sell_price": "mean"})
    .rename(columns={"avg_sell_price": "price"})
)

# Tạo thêm thông tin mô tả
dim_item["currency"] = "USD"
dim_item["dept_id"] = sales.groupby("item_id")["dept_id"].first().values
dim_item["cat_id"] = sales.groupby("item_id")["cat_id"].first().values
dim_item["item_name"] = dim_item["item_id"]  # giả định tạm item_name = item_id

# --- DIM_STORE ---
dim_store = sales[["store_id", "state_id"]].drop_duplicates().reset_index(drop=True)
dim_store["store_name"] = dim_store["store_id"]
dim_store["type"] = None
dim_store["size"] = None

# --- DIM_STATE ---
dim_state = sales[["state_id"]].drop_duplicates().reset_index(drop=True)
dim_state["state_name"] = dim_state["state_id"]
dim_state["region"] = None

# --- FACT_SALES ---
fact_sales = sales.merge(
    prices,
    on=["store_id", "item_id", "wm_yr_wk"],
    how="left"
)

fact_sales["revenue"] = fact_sales["weekly_units_sold"] * fact_sales["avg_sell_price"]
fact_sales = fact_sales.rename(columns={"wm_yr_wk": "week_id"})
fact_sales = fact_sales[[
    "week_id", "item_id", "store_id", "weekly_units_sold", "revenue"
]]

# --- SAVE ALL TO FILES ---
dim_calendar.to_csv(output_dir / "dim_calendar.csv", index=False)
dim_item.to_csv(output_dir / "dim_item.csv", index=False)
dim_store.to_csv(output_dir / "dim_store.csv", index=False)
dim_state.to_csv(output_dir / "dim_state.csv", index=False)
fact_sales.to_csv(output_dir / "fact_sales.csv", index=False)

print(f"✅ All star schema tables generated and saved to '{output_dir}' directory.")


TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [7]:
# === 2. Tiền xử lý cơ bản ===
# Kiểm tra null và loại bỏ nếu cần
calendar = calendar.dropna(subset=["wm_yr_wk"])
prices = prices.dropna(subset=["store_id", "item_id", "wm_yr_wk"])
sales = sales.dropna(subset=["item_id", "store_id", "state_id", "wm_yr_wk"])

# === 3. DIM TABLES ===

# --- dim_item ---
dim_item = (
    sales[["item_id", "dept_id", "cat_id"]]
    .drop_duplicates()
    .merge(prices.groupby("item_id")["avg_sell_price"].mean().reset_index(), on="item_id", how="left")
)
dim_item["item_name"] = dim_item["item_id"]  # giả định item_name = item_id
dim_item["price"] = dim_item["avg_sell_price"].round(2)
dim_item["currency"] = "USD"
dim_item = dim_item[["item_id", "dept_id", "cat_id", "item_name", "price", "currency"]]

# --- dim_store ---
dim_store = (
    sales[["store_id", "state_id"]]
    .drop_duplicates()
)
dim_store["store_name"] = dim_store["store_id"]
dim_store["type"] = "regular"
dim_store["size"] = np.random.randint(10000, 50000, len(dim_store))
dim_store = dim_store[["store_id", "store_name", "state_id", "type", "size"]]

# --- dim_state ---
dim_state = (
    sales[["state_id"]]
    .drop_duplicates()
)
dim_state["state_name"] = dim_state["state_id"].map({
    "CA": "California",
    "TX": "Texas",
    "WI": "Wisconsin"
}).fillna(dim_state["state_id"])
dim_state["region"] = dim_state["state_name"].map({
    "California": "West",
    "Texas": "South",
    "Wisconsin": "Midwest"
}).fillna("Unknown")

# --- dim_calendar ---
# Ở đây wm_yr_wk sẽ đại diện cho "week_id" thay cho "date_id"
dim_calendar = calendar.copy()
dim_calendar["date_id"] = dim_calendar["wm_yr_wk"]
dim_calendar["weekday"] = None
dim_calendar["month"] = None
dim_calendar["year"] = dim_calendar["wm_yr_wk"].astype(str).str[:4].astype(int)
dim_calendar["week_of_year"] = dim_calendar["wm_yr_wk"].astype(str).str[-2:].astype(int)
dim_calendar.rename(columns={
    "main_event_name": "event_name",
    "main_event_type": "event_type"
}, inplace=True)
dim_calendar["snap_CA"] = False
dim_calendar["snap_TX"] = False
dim_calendar["snap_WI"] = False
dim_calendar = dim_calendar[[
    "date_id", "weekday", "month", "year", "week_of_year",
    "event_name", "event_type", "snap_CA", "snap_TX", "snap_WI"
]]

# === 4. FACT TABLE ===
fact_sales = sales.merge(prices, on=["item_id", "store_id", "wm_yr_wk"], how="left")
fact_sales = fact_sales.merge(calendar, on="wm_yr_wk", how="left")

fact_sales["sales_id"] = range(1, len(fact_sales) + 1)
fact_sales["date_id"] = fact_sales["wm_yr_wk"]
fact_sales["units_sold"] = fact_sales["weekly_units_sold"]
fact_sales["revenue"] = fact_sales["weekly_units_sold"] * fact_sales["avg_sell_price"]

fact_sales = fact_sales[[
    "sales_id", "date_id", "item_id", "store_id", "units_sold", "revenue"
]]

# === 5. Lưu ra file CSV ===
dim_item.to_csv("dim_item.csv", index=False)
dim_store.to_csv("dim_store.csv", index=False)
dim_state.to_csv("dim_state.csv", index=False)
dim_calendar.to_csv("dim_calendar.csv", index=False)
fact_sales.to_csv("fact_sales.csv", index=False)

print("✅ Đã tạo và lưu 5 bảng star schema (dim + fact) thành công!")

✅ Đã tạo và lưu 5 bảng star schema (dim + fact) thành công!
